In [1]:
from google.colab import files
uploaded = files.upload()

Saving DATA_Ex1a.zip to DATA_Ex1a.zip


In [2]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['DATA_Ex1a.zip']), 'r')
data.extractall()

In [3]:
import os
import cv2
import numpy as np
import csv as csvD
import pandas as pd
import copy
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import shutil


In [4]:
dataframe = pd.read_csv(r"traza_REY.csv",sep=';',header=None)
dataframe.head(10)
dataframe.shape[0]

889

In [5]:
!mkdir annotations
!mkdir images

In [6]:
from pathlib import Path

rey_scan_folder = "/content/REY_scan_anonim"
annotacions_folder = "/content/annotations"

def imageExits(imageName):
  dirImage = rey_scan_folder + "/" + imageName
  my_file = Path(dirImage)  
  if my_file.is_file():
    return 1  
  return 0    

def annotationExits(imageName):
  imageNameForTxt = imageName.replace("png", "txt")
  imageNameForTxt = imageNameForTxt.replace("jpg", "txt")   
  dirImage = annotacions_folder + "/" + imageNameForTxt  
  my_file = Path(dirImage)  
  if my_file.is_file():
    return 1  
  return 0 


In [7]:
def CheckBBValues(imageName, bb_xSuperiorIzq, bb_ySuperiorIzq, 
                  bb_xInferiorDerecho, bb_yInferiorDerecho, x_center, y_center, height, width):
  
  dirImage = rey_scan_folder + "/" + imageName
  img1 = cv2.imread(dirImage)
  
  cv2.circle(img1,(bb_xSuperiorIzq,bb_ySuperiorIzq), 10, (0,0,255), -1)

  cv2.circle(img1,(bb_xInferiorDerecho,bb_yInferiorDerecho), 10, (0,0,255), -1)
  plt.imshow(img1)
  plt.show()

  #print rectangles
  cv2.rectangle(img1,(bb_xSuperiorIzq,bb_ySuperiorIzq),(bb_xInferiorDerecho,bb_yInferiorDerecho),(0,255,0),3)
  plt.imshow(img1)
  plt.show()

  cv2.circle(img1,(x_center,y_center), 10, (0,0,255), -1)
  plt.imshow(img1)
  plt.show()



In [8]:
def GetBBDataForYOLO(imageName, boundingBox):
  dirImage = rey_scan_folder + "/" + imageName  
  img1 = cv2.imread(dirImage)
  height, width = img1.shape[:2]

  boundingBoxValues = boundingBox.split(",")
  bb_xSuperiorIzq = int(boundingBoxValues[0].strip())  
  bb_ySuperiorIzq = int(boundingBoxValues[1].strip())
  bb_h = int(boundingBoxValues[2].strip())
  bb_w = int(boundingBoxValues[3].strip())

  #point inferior derecho
  bb_xInferiorDerecho = bb_xSuperiorIzq + bb_w
  bb_yInferiorDerecho = bb_ySuperiorIzq + bb_h

  #center point of bb
  x_center = round((bb_xInferiorDerecho + bb_xSuperiorIzq) / 2)
  y_center = round((bb_ySuperiorIzq + bb_yInferiorDerecho) / 2)

  #Check values --only for testing
  #CheckBBValues(imageName, bb_xSuperiorIzq, bb_ySuperiorIzq, bb_xInferiorDerecho, bb_yInferiorDerecho, x_center, y_center, height, width)
  
  #Normalizar valores para que estén entre 0 y 1
  xYolo = round(x_center / width, 6) 
  yYOLO = round(y_center / height, 6) 
  wYOLO = round(bb_w / width, 6) 
  hYOLO = round(bb_h / height, 6)

  return xYolo, yYOLO, wYOLO, hYOLO



In [9]:
def CalculateAndSaveAnnotation(imageName, xYolo, yYOLO, wYOLO, hYOLO):  
  class_id = 0 
  #Save
  print_buffer = []
  # Name of the file which we have to save 
  imageNameForTxt = imageName.replace("png", "txt")
  imageNameForTxt = imageNameForTxt.replace("jpg", "txt") 

  save_file_name = os.path.join("/content/annotations", imageNameForTxt)

  #Write the bbox details to the file 
  print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, xYolo, yYOLO, wYOLO, hYOLO))  

  # Save the annotation to disk
  print("\n".join(print_buffer), file= open(save_file_name, "w")) 
  


In [ ]:
for index, row in dataframe.iterrows():
  forig = row[2]
  bb = row[3]
  imageName = forig.replace("forig:", "").strip()
  boundingBox = bb.replace("BB:[", "")
  boundingBox = boundingBox.replace("]", "")  

  #comprobar que existe la imagen en el folder  
  exists = imageExits(imageName)
  existsAnn = annotationExits(imageName)
  if(exists == 1 and existsAnn == 0):
    xYolo, yYOLO, wYOLO, hYOLO = GetBBDataForYOLO(imageName, boundingBox)
    CalculateAndSaveAnnotation(imageName, xYolo, yYOLO, wYOLO, hYOLO)
  else:
    print('error')
  
  
    

In [11]:
# Read images and annotations
images = [os.path.join(rey_scan_folder, x) for x in os.listdir(rey_scan_folder)]
annotations = [os.path.join(annotacions_folder, x) for x in os.listdir(annotacions_folder)]
print(len(annotations))
print(len(images))

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

887
887


In [12]:
#Create the folders to keep the splits.
!mkdir images/train images/val images/test annotations/train annotations/val annotations/test

In [13]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

In [14]:
# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'annotations/train/')
move_files_to_folder(val_annotations, 'annotations/val/')
move_files_to_folder(test_annotations, 'annotations/test/')

In [15]:
# Rename annotations to labels
mv annotations labels

In [ ]:
#Zip dataset
!zip -r '/content/images.zip' '/content/images/'
!zip -r '/content/labels.zip' '/content/labels/'

In [17]:
from google.colab import files
files.download('/content/images.zip')
files.download('/content/labels.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>